In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from datetime import datetime
import time
from tqdm.notebook import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests
import json

s=requests.Session()
headers = {
    'authority': 'homes.hdb.gov.sg',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/plain, */*',
    'sessionid': 'a',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'content-type': 'application/json',
    'origin': 'https://homes.hdb.gov.sg',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://homes.hdb.gov.sg/home/finding-a-flat',
    'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
}

s.headers = headers

In [3]:
data = '{"location":"","coordinates":[["",""]],"range":"2","priceRangeLower":"0","priceRangeUpper":"0","flatType":"","waitingTime":"","modeOfSale":"BTO,SBF","floorRange":"","remainingLeaseRangeLower":1,"remainingLeaseRangeUpper":99,"ethnicGroup":"Chinese","rank":"Location, Price Range, Flat Type, Remaining Lease"}'

response = s.post('https://homes.hdb.gov.sg/home-api/public/v1/map/getCoordinatesByFilters', data=data)

response

<Response [200]>

In [4]:
jsonstr=json.loads(response.text)
detailedUrl='https://resources.homes.hdb.gov.sg/nf/{}.xml'
# https://resources.homes.hdb.gov.sg/nf/2021-05/bto/tg_d2c4/2021-05_BTO_VEdfRDJDNF8xNjE5NTAyODQxMDYz.xml
#all project details xml link follow this format - works well for BTO

for x in tqdm(jsonstr):
    # select only those launching
    x['properties']['description']=x['properties']['description'][0]
    if x['properties']['description']['stage']=='Launch':
        storageDict={}
    #     to try and reduce API calls on price range
        if x['properties']['listingType']=='BTO':
            projPath=x['properties']['description'].get('photoUrl','').rpartition('/')[0]
            newUrl=detailedUrl.format(projPath+'/'+x['properties']['description']['projectId'])
            details=s.get(newUrl)
        elif x['properties']['listingType']=='SBF':
            ballotQtr=x['properties']['description']['ballotQtr']
            projectID=x['properties']['description']['projectId']
            projDict='{"projectId":"'+projectID+'"}'
            storageDict={}
            x['availability']=json.loads(s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getProjectAvailabilityAndEthnic',data=projDict).text)
            for block,value in x['availability']['projectBlockFlatTypeInfoMap'].items():
                for flatType in value.keys():
                    if storageDict.get(flatType) is None:
                        priceDict='{"projectId":"'+projectID+'","flatType":"'+flatType+'"}'
                        storageDict[flatType]=s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getPriceRangeAcrossTownByProjectIdAndFlatType',data=priceDict).text
                    x['availability']['projectBlockFlatTypeInfoMap'][block][flatType]['areaPrice']=storageDict[flatType]
                        
            details  = s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getUnitXml',data=projDict)
        if details.status_code!=requests.codes.ok:
            print(x)
        x['details']=details.text
        # slowdown requests
        time.sleep(0.5)

  0%|          | 0/311 [00:00<?, ?it/s]

In [5]:
df=pd.json_normalize(jsonstr,max_level=1)
df.head(10)

,coordinates,details,properties.address,properties.listingType,properties.description,properties.hdbCategory,properties.region,availability.availabilitySet,availability.projectBlockFlatTypeInfoMap
0,"[1.2868908, 103.8237244]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Bukit Merah Ridge,BTO,{'photoUrl': '2022-05/bto/bm_n6c58/bm_n6c58_th...,1,CENTRAL REGION,NaN,NaN
1,"[1.3401562, 103.7226641]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Lakeside View,BTO,{'photoUrl': '2022-05/bto/jw_n3c31/jw_n3c31_th...,1,WEST REGION,NaN,NaN
2,"[1.3115339, 103.7898642]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Ghim Moh Ascent,BTO,{'photoUrl': '2022-05/bto/qt_n6c8/qt_n6c8_thum...,1,CENTRAL REGION,NaN,NaN
3,"[1.338306, 103.8548273]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Kim Keat Heights,BTO,{'photoUrl': '2022-05/bto/tp_n1c27/tp_n1c27_th...,1,CENTRAL REGION,NaN,NaN
4,"[1.4238382, 103.8359392]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Yishun Beacon,BTO,{'photoUrl': '2022-05/bto/ys_n9c3/ys_n9c3_thum...,1,NORTH REGION,NaN,NaN
5,"[1.3766745369726854, 103.73789561376547]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Sunshine Court,SBF,"{'maxRemainingLease': '45', 'stage': 'Launch',...",2,WEST REGION,"[2022-05_SBF_476A_09_07, 2022-05_SBF_476A_02_01]","{'476A': {'2-room flexi': {'block': '476A', 'f..."
6,"[1.3765873889910192, 103.74007753792313]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Sunshine Gardens,SBF,"{'maxRemainingLease': '94', 'stage': 'Launch',...",2,WEST REGION,"[2022-05_SBF_488C_02_163, 2022-05_SBF_488C_02_...","{'488C': {'5-room': {'block': '488C', 'flatTyp..."
7,"[1.3942738802398278, 103.74452794822368]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Limbang Green,SBF,"{'maxRemainingLease': '45', 'stage': 'Launch',...",2,WEST REGION,"[2022-05_SBF_573_08_280, 2022-05_SBF_571_04_24...","{'573': {'2-room flexi': {'block': '573', 'fla..."
8,"[1.394688561612174, 103.74734039850861]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Choa Chu Kang Dr,SBF,"{'maxRemainingLease': '74', 'stage': 'Launch',...",2,WEST REGION,[2022-05_SBF_784_03_207],"{'784': {'4-room': {'block': '784', 'flatType'..."
9,"[1.3755267556066781, 103.74967816227914]","<?xml version=""1.0"" encoding=""UTF-8"" standalon...",Keat Hong Verge,SBF,"{'maxRemainingLease': '99', 'stage': 'Launch',...",2,WEST REGION,"[2022-05_SBF_819A_10_316, 2022-05_SBF_819A_21_...","{'820B': {'3-room': {'block': '820B', 'flatTyp..."


In [6]:
df['details'][5]

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><template><project-name>Sunshine Court</project-name><standard-project-info><flat-types><flat-type><type>2-Room Flexi</type><block><block-number>476A</block-number><unit-id-list><unit-id>2022-05_SBF_476A_02_01</unit-id><unit-id>2022-05_SBF_476A_09_07</unit-id></unit-id-list><note>Mouseover to see prices for all lease periods.</note></block></flat-type></flat-types><ethnics><available-ethnic><ethnic>Malay</ethnic><block><block-number>476A</block-number><unit-id-list><unit-id>2022-05_SBF_476A_02_01</unit-id><unit-id>2022-05_SBF_476A_09_07</unit-id></unit-id-list></block></available-ethnic><available-ethnic><ethnic>Chinese</ethnic><block><block-number>476A</block-number><unit-id-list><unit-id>2022-05_SBF_476A_02_01</unit-id><unit-id>2022-05_SBF_476A_09_07</unit-id></unit-id-list></block></available-ethnic><available-ethnic><ethnic>Indian/Other Races</ethnic><block><block-number>476A</block-number><unit-id-list><unit-id>2022-05_SBF_47

In [7]:
# create past data save path, use balot qtr 
monthYear = datetime.strptime(jsonstr[0]['properties']['description']['ballotQtr'],'%Y-%m').strftime('%h-%Y').lower()
year = datetime.now().strftime('%Y')
savePath = os.path.join('past_data', year, monthYear)
Path(savePath).mkdir(parents=True, exist_ok=True)

print(savePath)

past_data\2022\may-2022


In [8]:
# save data in various formats
fileName = os.path.join(savePath, 'hdb-sale-'+monthYear)
df.to_hdf(fileName+'.h5','HDBSALES')
df.to_pickle(fileName+'.pkl')
with open(fileName+".json", "w") as write_file:
    json.dump(jsonstr, write_file)

C:\Users\willi\miniconda3\envs\workspace\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['coordinates', 'details', 'properties.address',
       'properties.listingType', 'properties.description',
       'properties.hdbCategory', 'properties.region',
       'availability.availabilitySet',
       'availability.projectBlockFlatTypeInfoMap'],
      dtype='object')]

  pytables.to_hdf(


In [9]:
jsonstr[1]

{'coordinates': '[1.3401562, 103.7226641]',
 'properties': {'address': 'Lakeside View',
  'listingType': 'BTO',
  'description': {'photoUrl': '2022-05/bto/jw_n3c31/jw_n3c31_thumbnail.jpg',
   'maxRemainingLease': '99',
   'stage': 'Launch',
   'distance': '-1',
   'town': 'Jurong West',
   'launchStartDate': '2022-05-27 10:00:00.0',
   'ballotQtr': '2022-05',
   'flatType': '2-Room Flexi, 3-Room (Income Ceiling $7,000), 4-Room, 5-Room',
   'maxPrice': '526000',
   'projectId': '2022-05_BTO_SldfTjNDMzFfMTY1MDI3NDIwODU5Nw'},
  'hdbCategory': '1',
  'region': 'WEST REGION'},
 'details': '<?xml version="1.0" encoding="UTF-8"?>\n<template>\n    <project-name>Lakeside View</project-name>\n    <standard-project-info>\n        <key>\n            <town>Jurong West</town>\n            <tooltip/>\n            <uri/>\n        </key>\n        <key>\n            <point>1231|1283</point>\n        </key>\n        <key>\n            <latitude>1.3401562</latitude>\n            <longitude>103.7226641</lo